In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model 
import pandas as pd
import openpyxl
import sklearn
import datetime
from tensorflow.keras.utils import to_categorical
import itertools
from tensorflow.keras.applications import VGG19, ResNet50V2, MobileNet, DenseNet121, InceptionV3
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from keras.models import Model

In [ ]:
inp = layers.Input(shape=(224, 224, 3))
dataset_name = 'Dataset1'
pre_traind_base = DenseNet121(include_top = False, 
                                weights='imagenet', input_tensor=inp,
                                input_shape=[224,224, 3])
pre_traind_base.trainable = False




In [ ]:
categories = ["1","2","3","4"]
num_classes = len(categories)
img_rows, img_cols = 224, 224
X_train, X_test, Y_train, Y_test = np.load(f'D:/article/npy/{dataset_name}.npy', allow_pickle = True)
epoch = 20
savename = f"D:/article/result/Mobile_{dataset_name}_{epoch}/"


In [ ]:
def build_cnn_model():
    model = models.Sequential()
    model.add(layers.Conv2D(16, (3, 3), strides=(1,1), input_shape=(224,224, 3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides = (2,2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides = (2,2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides = (2,2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides = (2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation = 'relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation = 'softmax'))
    return model

In [ ]:
def build_trainsfer_classifier():
    flat1 = layers.Flatten()(pre_traind_base.layers[-1].output)
    x = layers.Dense(128, activation='relu')(flat1)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(num_classes, activation='softmax')(x)
    model =  Model(inputs=pre_traind_base.inputs, outputs=output)
    return model

In [ ]:
model = build_trainsfer_classifier()
model.summary()

In [ ]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )
start_time = datetime.datetime.now()
history = model.fit(X_train, Y_train, epochs=epoch, validation_split=0.2 )
end_time = datetime.datetime.now()
elapsed_time = end_time - start_time
print(f"training time: {elapsed_time} sec")
_, acc = model.evaluate(X_test, Y_test)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

In [ ]:
start_time2 = datetime.datetime.now()
y_pred = model.predict(X_test) 
end_time2 = datetime.datetime.now()
elapsed_time2 = end_time2 - start_time2
print(f"test time: {elapsed_time2} sec")
print(f"training time: {elapsed_time} sec")



In [ ]:
weight_pred = y_pred
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(Y_test, axis=1)
matric = sklearn.metrics.confusion_matrix(y_test, y_pred, labels=None, sample_weight=None, normalize=None)
weight = pd.DataFrame(weight_pred)
weight['y_pred'] = y_pred
weight['y_true'] = y_test
hist = pd.DataFrame(history.history)
matric = pd.DataFrame(matric)
weight = pd.DataFrame(weight)


model.save(f"{savename}model.h5")
hist.to_excel(f"{savename}history.xlsx")
matric.to_excel(f"{savename}matric.xlsx")
weight.to_excel(f"{savename}weight.xlsx")
with open(f"{savename}time.txt", "w") as f:
    f.write(f"training: {elapsed_time}\ntest: {elapsed_time2}")

In [ ]:
matric = sklearn.metrics.confusion_matrix(y_test, y_pred, labels=None, sample_weight=None, normalize=None)
plt.imshow(matric, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(num_classes)
# plt.xticks(tick_marks, ['high grade','third', 'out of'])
# plt.yticks(tick_marks, ['high grade','third', 'out of'])
plt.xticks(tick_marks, ['first', 'second','third', 'out of'])
plt.yticks(tick_marks, ['first', 'second','third', 'out of'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
thresh = matric.max() / 2.
for i, j in itertools.product(range(matric.shape[0]), range(matric.shape[1])):
    plt.text(j, i, format(matric[i, j], 'd'),
             horizontalalignment="center",
             color="white" if matric[i, j] > thresh else "black")
plt.savefig(f"{savename}matric.jpg")

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
print('Accuracy: {:.4f}'.format(accuracy))
print('Precision: {:.4f}'.format(precision))
print('Recall: {:.4f}'.format(recall))
print('F1 score: {:.4f}'.format(f1))
with open(f"{savename}performance.txt", "w") as f:
    f.write(f"Accuracy: {round(accuracy, 4)}\nPrecision: {round(precision, 4)}\nRecall: {round(recall, 4)}\nF1_score: {round(f1, 4)}")


In [ ]:
y_test_cat = to_categorical(y_test, num_classes=num_classes)
y_pred_cat = to_categorical(y_pred, num_classes=num_classes)
# Calculate ROC AUC score
auc = roc_auc_score(y_test_cat, y_pred_cat, multi_class='ovo')

# Calculate FPR and TPR values for ROC curve
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_cat[:, i], y_pred_cat[:, i])
    roc_auc[i] = roc_auc_score(y_test_cat[:, i], y_pred_cat[:, i])
# Plot the ROC curve
plt.figure()
colors = ['aqua', 'darkorange', 'cornflowerblue', 'red']
# colors = ['violet', 'cornflowerblue', 'red']
for i, color in zip(range(num_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of {0} grade (AUC = {1:0.4f})'.format(['first', 'second', 'third', 'out of'][i], roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.savefig(f"{savename}AUC.jpg")
print('AUC: {:.4f}'.format(auc))
